# 動作確認用のファイル

In [99]:
import pandas as pd
import math

In [2]:
tanka = pd.read_csv("kindai.csv",header=None)
waka = pd.read_csv("waka_half.csv",header=None)

In [3]:
print(tanka.info())
print(tanka.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13963 entries, 0 to 13962
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       13963 non-null  object
dtypes: object(1)
memory usage: 109.2+ KB
None
                                     0
0  あああらき なげきやながき くろかみも ひとときにして あせやしぬべし
1  あああらき なげきやなんじ れはかりは てんもうごけと なくにもにたる
2  ああきみは いたましきまで ようちなる なさけにゑひぬ かなしからずや
3  ああきみは かたるにたらず よわきじん よわきこいをば もてあそぶかな
4  ああきみは すむむねもなき さびしさを かたるかなみだ かわきしめもて


In [4]:
print(waka.info())
print(waka.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14292 entries, 0 to 14291
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       14292 non-null  object
dtypes: object(1)
memory usage: 111.8+ KB
None
                                     0
0  あかからは みるへきものを かりかねの いつこはかりに なきてゆくらむ
1  あかさりし おもかけはかり うつしても ちきりはさてや やまのゐのみつ
2  あかさりし きみかにほひの こひしさに うめのはなをそ けさはをりつる
3  あかさりし そてかとにほふ うめかかに おもひなくさむ あかつきのそら
4  あかさりし そてのわかれの なみたより かたみかほなる つきそみにそふ


In [131]:
def vectorize_df(df):
    list_df = df.apply(split_text,axis=1)
    tri_gram_df = list_df.apply(tri_gram)
    vec_id_dic = calc_vector(tri_gram_df)
    out_df = convert_to_vector(tri_gram_df,vec_id_dic)
    return out_df

def split_text(text_record):
    return text_record.loc[0].split(" ")

def tri_gram(text_list):
    ans = []
    for text in text_list:
        gram = []
        for i in range(len(text)-2):
            gram.append(text[i:i+3])
        ans.extend(gram)
    return ans

def calc_vector(gram_df):
    vector = {}
    vector["vector"] = {}
    vector["id"] = {}
    r2 = 0
    id = 0
    for line in gram_df:
        for word in line:
            if word in vector["vector"].keys():
                vector["vector"][word] += 1
            else:
                vector["vector"][word] = 1
                vector["id"][word] = id
            id += 1

    for key in vector["vector"].keys():
        r2 += vector["vector"][key]**2

    r = math.sqrt(r2)
    for key in vector["vector"].keys():
        vector["vector"][key] /= r

    return vector

def convert_to_vector(df,dic):
    out_vec_list = []
    out_id_list = []
    out_df = pd.DataFrame()
    for i in range(df.shape[0]):
        vec_list = []
        id_list = []
        word_list = df.loc[i]
        for word in word_list:
            vec_list.append(dic["vector"][word])
            id_list.append(dic["id"][word])
        out_vec_list.append(vec_list)
        out_id_list.append(id_list)
    out_df["vector"] = out_vec_list
    out_df["id"] = out_id_list
    return out_df


In [138]:
waka_vector_id = vectorize_df(waka)
tanka_vector_id = vectorize_df(tanka)

In [139]:
waka_path = "waka_vector.csv"
tanka_path = "tanka_vector.csv"

waka_vector_id.to_csv(waka_path,index=False)
tanka_vector_id.to_csv(tanka_path,index=False)

In [140]:
pd.read_csv(waka_path)

,vector,id
0,"[0.00018928903873745214, 0.009275162898135155,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"[0.0030286246197992342, 0.004353647890961399, ...","[21, 22, 23, 24, 25, 26, 27, 14, 29, 30, 31, 3..."
2,"[0.0030286246197992342, 0.004353647890961399, ...","[21, 22, 23, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
3,"[0.0030286246197992342, 0.004353647890961399, ...","[21, 22, 23, 66, 67, 68, 69, 70, 71, 72, 73, 7..."
4,"[0.0030286246197992342, 0.004353647890961399, ...","[21, 22, 23, 87, 88, 89, 90, 91, 92, 93, 94, 9..."
...,...,...
14287,"[0.0007571561549498086, 0.016468146370158334, ...","[43620, 405, 893, 48, 23676, 221303, 172374, 5..."
14288,"[0.0007571561549498086, 0.016468146370158334, ...","[43620, 405, 6269, 6586, 126710, 58797, 1904, ..."
14289,"[0.0005678671162123564, 0.023661129842181515, ...","[53855, 4479, 427, 36359, 955, 5353, 135, 136,..."
14290,"[0.0005678671162123564, 0.023661129842181515, ...","[53855, 4479, 4480, 35682, 35683, 1528, 1529, ..."
